# FEATURE SELECTION (Improve model accuracy, lower computational cost, easier to understand & explain)


SUPERVISED
+ Intrinsic: 
+ Wrapper Method (Bao bọc): Forward FS, Backward FS, Exhaustive FS, RFECV
+ Filter Method (less time, not overfitting): Infomation gain, Chi2, Fisher's Score, Missing value ratio
 1. Correlation Statistics: biến độc lập tương quan cao với biến phụ thuộc thì không nên loại bỏ, các biến độc lập tương quan cao với nhau nên loại bỏ.
 2. Selection Method:
 3. Transform Variables:
+ Embedded (giống filter nhưng chính xác hơn): L1, Elastic(L1+L2), random forest importance


REGRESSION
1. INPUT là NUMERICAL:
+ Pearson's (linear): f_regression()
+ Spearman's (non-linear): scipy.stats.spearmanr()
2. INPUT là CATEGORICAL:
+ Kendall's (non-linear: giả định categorical là ordinal): scipy.stats.kendalltau(X,y)
+ Anova (linear): f_classif
+ Mutual information(regression)

CLASSIFICATION
1. Input là NUMERICAL:
+ Kendall's (non-linear): giả định categorical là ordinal): scipy.stats.kendalltau(X,y)
+ Anova (linear): f_classif
2. Input là CATEGORICAL:
+ Chi_squared (ưu tiên)
+ Mutual information (classification)

# MUTUAL INFORMATION IN REGRESSION PROBLEM STATEMENTS
Selection-Information gain - mutual information In Classification Problem Statements
Mutual Information
MI Estimate mutual information for a discrete target variable.

Mutual information (MI) between two random variables is a non-negative value, which measures the dependency between the variables. It is equal to zero if and only if two random variables are independent, and higher values mean higher dependency.

The function relies on nonparametric methods based on entropy estimation from k-nearest neighbors distances.

Inshort:
A quantity called mutual information measures the amount of information one can obtain from one random variable given another.

The mutual information between two random variables X and Y can be stated formally as follows:

I(X ; Y) = H(X) - H(X | Y) Where I(X ; Y) is the mutual information for X and Y, H(X) is the entropy for X and H(X | Y) is the conditional entropy for X given Y. The result has the units of bits.

Youtube Videos
+ Entropy: https://www.youtube.com/watch?v=1IQOtJ4NI_0
+ Information Gain: https://www.youtube.com/watch?v=FuTRucXB9rA
+ Gini Impurity: https://www.youtube.com/watch?v=5aIFgrrTqOw
+ Statistical test: https://www.youtube.com/watch?v=4-rxTA_5_xA

In [2]:
import pandas as pd,numpy as np,matplotlib.pyplot as plt,seaborn as sns,warnings
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.feature_selection import SelectFromModel,SelectKBest,VarianceThreshold,RFECV,mutual_info_classif
from sklearn.ensemble import ExtraTreesClassifier
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df = pd.read_csv(r"C:\Users\ADMIN\Documents\Data_Science\Data\House_price_train.csv")
X_train = df.drop(columns=['Id','SalePrice'],axis=1)
y_train = df[['SalePrice']]

# SUPERVISED
# 1. Embedded

1.1. LASSO 

In [12]:
def FS_Lasso(X,y,alpha):
    feature_select = SelectFromModel(Lasso(alpha=alpha,random_state=2))
    feature_select.fit(X,y)
    select_feature = X.columns[feature_select.get_support()]
    X_lasso = X[select_feature]
    return X_lasso

In [14]:
X_train = FS_Lasso(X_train,y_train,alpha=0.005)

1.2. ENSEMBLE : càng cao càng quan trọng, có quan hệ chặt chẽ với y.

In [ ]:
def FS_ExtraTrees(X,y,num_feature):
    model = ExtraTreesClassifier()
    model.fit(X,y)
    ranker_features = pd.Series(model.feature_importances_,index=columns)
    ranker_features.nlargest(num_feature).plot(kind='barh')
    X_extre_trees = X[ranker_features.nlargest(num_feature).index]
    return X_extre_trees

In [ ]:
X_train = FS_ExtraTrees(X_train,y_train,num_feature=10)

# 2. FILTER METHOD

2.1. CORRELATION

In [ ]:
# # CORRELATION
# def plot_corr(df,plot_name):
#     data = df.corr().iloc[1:,:-1]
#     mask = np.triu(np.ones_like(df.corr(),dtype=bool))[1:,:-1]
#     cmap = sns.diverging_palette(0,230,90,60,as_cmap=True)
#     fig,ax = plt.subplots(figsize=(16,8))
#     sns.heatmap(data=data,cmap=cmap,mask=mask,annot=True,annot_kws={'fontsize':13},fmt='.2f',vmin=-1,vmax=1,
#                 linecolor='white',linewidths=.5)
#     ax.set_yticklabels([i.upper() for i in data.index],rotation=0)
#     ax.set_xticklabels([i.upper() for i in data.columns],rotation=0)
#     ax.set_title(plot_name,loc='right',fontsize=24)
#     ax.set_ylabel('')
#     ax.set_xlabel('')
#     fig.tight_layout()
# plot_corr(X_train,'CORRELATION MATRIX BANK MARKETING')
# def Correlation(dataset, threshold):
#     col_corr = set()
#     corr_matrix = dataset.corr()
#     for i in range(len(corr_matrix.columns)):
#         for j in range(i):
#             if abs(corr_matrix.iloc[i, j]) > threshold: 
#                 colname = corr_matrix.columns[i]
#                 col_corr.add(colname)
#     return col_corr
# corr_features = Correlation(X_train,threshold = 0.7)
# X_train = X_train.drop(columns = corr_features,axis=1)

In [12]:
def plot_corr(df,plot_name):
    data = df.corr().iloc[1:,:-1]
    mask = np.triu(np.ones_like(df.corr(),dtype=bool))[1:,:-1]
    cmap = sns.diverging_palette(0,230,90,60,as_cmap=True)
    fig,ax = plt.subplots(figsize=(30,20))
    sns.heatmap(data=data,cmap=cmap,mask=mask,annot=True,annot_kws={'fontsize':13},fmt='.2f',vmin=-1,vmax=1,linecolor='white',linewidths=.5)
    ax.set_yticklabels([i.upper() for i in data.index],rotation=0)
    ax.set_xticklabels([i.upper() for i in data.columns],rotation=0)
    ax.set_title(plot_name,loc='right',fontsize=20)
    plt.tight_layout()

In [14]:
plot_corr(X_train,'CORRELATION')

In [ ]:
def Correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: 
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [3]:
corr_features = Correlation(X,threshold = 0.7)

In [2]:
X_train.drop(corr_features,axis=1,inplace=True)

2.2 SELECTION METHOD: điểm càng cao càng tốt

In [ ]:
def Univariate(X_train,y_train,X_test,method,num_feature):
    ordered_rank_features = SelectKBest(score_func=f_classif,k=5)
    df_train = pd.DataFrame(ordered_rank_features.fit_transform(X_train_scale,y_train),columns=ordered_rank_features.get_feature_names_out())
    df_test =  X_test[ordered_rank_features.get_feature_names_out()]
    return df_train,df_test

In [24]:
df_train,df_test = Univariate(X_train_scale,y_train,X_valid_scale,f_classif,num_feature=5)

In [5]:
# select_top_columns = SelectPercentile(mutual_info_regression,percentile=20).fit(x_train,y_train)
# df_mutual_info_reg = pd.concat([x_train[x_train.columns[select_top_columns.get_support()]],y_train],axis=1)
# mutual_info_classification = mutual_info_classif(x_train,y_train)
# mutual_info_classification = pd.Series(mutual_info_classification,index=x_train.columns)
# mutual_info_classification.sort_values(ascending=True).plot.bar(figsize=(15,5))

In [ ]:
# mutual_info_classification = mutual_info_classif(Train_scale,y_train)
# mutual_info_classification = pd.Series(mutual_info_classification,index=Train_scale.columns)
# mutual_info_classification.sort_values(ascending=True).plot.bar(figsize=(15,5))
# Train_scale = Train_scale[mutual_info_classification.sort_values(ascending=False).index[0:22]]
# test_scale = test_scale[mutual_info_classification.sort_values(ascending=False).index[0:22]]

# WRAPPER METHOD
# 1. RECURSIVE FEATURE ELIMINATION WITH CROSS VALIDATION

In [ ]:
def RFECV(X,y):
    regressor = LinearRegression()
    feature_selector = RFECV(regressor)
    fit = feature_selector.fit(X,y)
    optimal_feature_count = feature_selector.n_features_
    print(f"Optimal numer of features: {optimal_feature_count}")
    X_new = X.loc[:, feature_selector.get_support()]
    return X_new

In [ ]:
# RFECV(X,y)

In [ ]:
# fig,ax = plts.subplots()
# ax.plot(range(1, len(fit.grid_scores_) + 1), fit.grid_scores_, marker = "o")
# ax.set(ylabel='Model Score',xlabel='Number of Features')
# ax.set_title(f"Feature Selection using RFE \n Optimal number of features is {optimal_feature_count} (at score of {round(max(fit.grid_scores_), 4)})")
# fight.tight_layout() 

# UNSUPERVISED
# DROPPING CONSTANT FEATURES

In [25]:
def Var_Threshold(X,threshold):
    var_thresold = VarianceThreshold(threshold=threshold)
    var_thresold.fit(X)
    select_feature_var = X.columns[var_threshold.get_support()]
    constant_columns = [feature for feature in X.columns if feature not in selet_feature_var]
    X_drop_constant = X.drop(columns=constant_columns)
    return X_drop_constant

In [ ]:
# X = Var_Threshold(X,threshold=0.6)